<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training_vgg16_bn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

In [5]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 136 (delta 27), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (136/136), 6.05 MiB | 13.47 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/cloned-repo/cloned-repo
dataset.py		    EAST_resnet.py		  model.py
detect.py		    EAST_training.ipynb		  model_resnet.py
dsiplay_ground_truth.ipynb  EAST_training_vgg16_bn.ipynb  train.py
EAST_detection.ipynb	    loss.py
CPU times: user 90.7 ms, sys: 26.5 ms, total: 117 ms
Wall time: 6.01 s


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from dataset import *

In [15]:
# AI4Bharat dataset
image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Image/23.zip (Unzipped Files)'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotation/23_labels.zip (Unzipped Files)'

print('Total image files',len(os.listdir(image_address)))
print('Total annotation files',len(os.listdir(label_address)))

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

Total image files 4669
Total annotation files 4669
4669


## Dataset Initial Check

In [16]:
img = Image.open(image_address+'/0.jpg')

with open(label_address+'/0.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[ 18.92597422 188.22003706  62.99098946 187.28137344  63.44860586
 208.78910501  19.38359062 209.72776863]
--------------------
[ 19.52087554 216.1800881   77.55772488 214.94379943  77.87805636
 229.99921153  19.84120702 231.2355002 ]
--------------------
[175.15592034 304.31407172 198.77768184 302.73501198 201.77378512
 342.47852692 177.07726715 344.08048089]
--------------------
[  9.16228479 234.69060228 141.3573305  231.87461142 142.3869674
 280.26700746  10.19192169 283.08299831]
--------------------
[1 1 1 1]


In [17]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(247.)

## Loss for one batch

In [18]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=8)

imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))

print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())

print(torch.sum(gt_score))

torch.Size([8, 3, 512, 512]) torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128]) torch.Size([8, 1, 128, 128])
tensor(2595.)
CPU times: user 3.4 s, sys: 3.83 s, total: 7.23 s
Wall time: 9.59 s


In [19]:
from model import EAST
model = EAST()
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

Downloading: "http://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:15<00:00, 34.7MB/s]


torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128])


In [21]:
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128])


In [20]:
from loss import Loss

loss_fn = Loss()

loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.95671529, angle loss is 0.71677685, iou loss is 3.36248040


tensor(11.4870, grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
model = EAST()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrained_path = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/model_epoch_295.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=epoch_iter//2, gamma=0.1)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, 'model_epoch_{}.pth'.format(epoch+1))
      
      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))
    return loss_curve

In [0]:
%%time
loss_curve = train(model, image_address, label_address, batch_size=12, lr=1e-4, epoch_iter=200, interval = 50)

4669


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.45080006, angle loss is 0.01200036, iou loss is 0.82583576
Epoch is [1/200], mini-batch is [1/389], time consumption is 2.06036758, batch_loss is 1.39663935
classify loss is 0.36339825, angle loss is 0.01754026, iou loss is 0.71575230
Epoch is [1/200], mini-batch is [2/389], time consumption is 1.20160317, batch_loss is 1.25455308
classify loss is 0.40069693, angle loss is 0.01261010, iou loss is 0.73329240
Epoch is [1/200], mini-batch is [3/389], time consumption is 1.10073304, batch_loss is 1.26009035
classify loss is 0.33830237, angle loss is 0.02135983, iou loss is 0.51891071
Epoch is [1/200], mini-batch is [4/389], time consumption is 1.03928757, batch_loss is 1.07081127
classify loss is 0.24933201, angle loss is 0.01800870, iou loss is 0.44466269
Epoch is [1/200], mini-batch is [5/389], time consumption is 1.01978493, batch_loss is 0.87408167
classify loss is 0.43397623, angle loss is 0.02156006, iou loss is 0.75013387
Epoch is [1/200], mini-batch is [6/389], t

In [0]:
plt.plot(loss_curve)

[Adam and AdamW algorithms](https://www.fast.ai/2018/07/02/adam-weight-decay/)